In [1]:
# Download the Search Engine Code
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-27 11:23:32--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-27 11:23:32 (14.1 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [3]:
# Download the documents
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-27 11:24:53--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.009s  

2024-06-27 11:24:54 (66.1 MB/s) - ‘documents.json’ saved [658332/658332]



In [1]:
# Importing Search Engine library from the downloaded file
import minsearch

In [2]:
import json

In [3]:
# Read the documents from JSON format
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
# Get rid of nested structure in documents
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
# Create an index on the data
# Keywords fields are used for filtering (we may want to search only documents for data engineering zoomcamp)
# Text fields are the ones used for the actual search
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
# Determining the query we want to do on our data?
q = "The course has already started, can I still enroll?"

In [7]:
# Fit the index to the documents
index.fit(documents)

In [8]:
# Searching documents with relevant info for the query
# Boost is used when you think one of the fields is more important for the answer (the question field is 3x more important than the text field)
boost = {'question': 3, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [9]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [10]:
from openai import OpenAI

In [11]:
client = OpenAI()

In [12]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content':q}]
)

In [13]:
response.choices[0].message.content

"The ability to enroll in a course that has already started largely depends on the specific policies of the institution or the platform offering the course. Here are some general tips:\n\n1. **Contact the Instructor or Institution**: Reach out directly to the course instructor or the administrative office. They may allow late enrollment and provide the necessary guidance on how to catch up with missed work.\n\n2. **Check Enrollment Policies**: Sometimes, the enrollment policies are outlined on the institution's website. Look for information on late registration deadlines and procedures.\n\n3. **Online Platforms**: If the course is offered through an online learning platform (like Coursera, Udemy, etc.), they often have flexible enrollment options. Check the course page for details.\n\n4. **Self-Paced Courses**: If the course is self-paced, you can usually enroll at any time and complete the work according to your schedule.\n\n5. **Financial or Administrative Deadlines**: Be mindful of 

In [19]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [20]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [23]:
prompt = prompt_template.format(question=q, context=context).strip()

In [24]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content':prompt}]
)

response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."